# Capstone Project: Social media sentiment analysis

## Part 3c: Topic Modeling using Biterm

In [1]:
# Download biterm model from following github
#!pip install git+git://github.com/markoarnauto/biterm.git

In [8]:
import pandas as pd
import numpy as np
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

from biterm.btm import oBTM
from biterm.utility import vec_to_biterms, topic_summuary

In [22]:
# Call data from Part 2
tweet_combined_clean=pd.read_csv('./dataset/tweet_combined_clean_v1.csv')

In [5]:
# Unlike VADER, topic modeling requires extensive data cleaning - Edit stopwords to exclude emotional words like 'like','love' and brand names 'Samsung', 'Apple', 'Huawei'

lemm = WordNetLemmatizer()

def text_processer(text):
    
    # 1. Remove HTML.
    review_text = BeautifulSoup(text).get_text()
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. Convert to lower case, split into individual words.
    words = letters_only.lower().split()
        
    # 4. Remove stop words
    stop_words = set(stopwords.words("english"))
    stop_words.update(['samsung','huawei','apple','http','like','really','want','good','say','love','better','much',
                       'day','lol','well','need','could','take','twt','add','maxwinebach','nhlblackhawk','smtshepossa',
                       'niantichelp','recognised','view','great','doh','samsungmobile','okay','check','feel','always',
                       'yes','stop','even','every','already','u','something','go','see','sure','shit','said','https','com','www','hi','please','co','thanks','one',
                       'think','got','also', 'make','know','use', 'would','get','look','never','still','mtshepossa','akinjoshua',
                       'pay','using','time','b','c','d','e','f','g','h','i','ksvaljek', 'right', 'used','godissfroot','nhlblackhawks','http',
                       'j','k','l','n','m','o','p','q','r','s','t','u','v','w','x','y','z', 'sorry', 'part', 'u', 'let','as', 'saying', 'bit', 
                       'update', 'techquotesdaily','oh', 'yeah','frecowang', 'bts', 'pak','ok','fuck','come','thing','south','settle','level', 
                       'took','actually','stand','im','watch', 'jezdez','offby','dirtytesla','universeice','as', 'thank', 'thanks', 'seem', 'seems',
                        'way','put','made','thought', 'jack'])
    filtered_words=[w for w in words if not w in stop_words]

    # 5. Lemmatize words.
    lemmed_words = [lemm.lemmatize(i) for i in filtered_words]
    
    return (" ".join(lemmed_words))

In [6]:
# Initialise both types of vectorizer for comparison
cv = CountVectorizer(analyzer='word',       
                             min_df=10,
                            token_pattern='[a-zA-Z0-9]{3,}') #set number of characters to be more than 3 per word

## Samsung

In [32]:
tweet_samsung_clean=tweet_combined_clean[tweet_combined_clean['brand']==0]
tweet_samsung_words=tweet_samsung_clean['text'].apply(text_processer)
samsung_cv=cv.fit_transform(tweet_samsung_words).toarray()

In [33]:
vocab = np.array(cv.get_feature_names())
biterms = vec_to_biterms(samsung_cv)

btm = oBTM(num_topics=2, V=vocab)
for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
    biterms_chunk = biterms[i:i + 100]
    btm.fit(biterms_chunk, iterations=50)
    topics = btm.transform(biterms)
    
print("\n\n Topic coherence ..")
topic_summuary(btm.phi_wz.T, samsung_cv, vocab, 15)

100%|██████████| 50/50 [00:00<00:00, 136.30it/s]




 Topic coherence ..
Topic 0 | Coherence=-260.11 | Top words= samsungmobileus samsunguk huaweiuk huaweimobileuk zoneoftech prob zareldo pro phone galaxy brand samsungus shot channel iphone
Topic 1 | Coherence=-365.49 | Top words= phone galaxy iphone note android app new oneplus plus last camera smart mobile screen year
Average topic coherence for the top words is -312.80393944350476


{'coherence': [-260.11309567426633, -365.49478321274324],
 'top_words': [array(['samsungmobileus', 'samsunguk', 'huaweiuk', 'huaweimobileuk',
         'zoneoftech', 'prob', 'zareldo', 'pro', 'phone', 'galaxy', 'brand',
         'samsungus', 'shot', 'channel', 'iphone'], dtype='<U15'),
  array(['phone', 'galaxy', 'iphone', 'note', 'android', 'app', 'new',
         'oneplus', 'plus', 'last', 'camera', 'smart', 'mobile', 'screen',
         'year'], dtype='<U15')]}

In [34]:
count_0=0
for i in range(len(tweet_samsung_words)):
    if topics[i].argmax()==0:
        count_0+=1
print('Number of tweets under Topic 0: ', count_0)
count_1=0
for i in range(len(tweet_samsung_words)):
    if topics[i].argmax()==1:
        count_1+=1
print('Number of tweets under Topic 1: ', count_1)

Number of tweets under Topic 0:  1365
Number of tweets under Topic 1:  1073


## Apple

In [35]:
tweet_apple_clean=tweet_combined_clean[tweet_combined_clean['brand']==1]
tweet_apple_words=tweet_apple_clean['text'].apply(text_processer)
apple_vectorized=cv.fit_transform(tweet_apple_words).toarray()

In [36]:
vocab = np.array(cv.get_feature_names())
biterms = vec_to_biterms(apple_vectorized)

btm = oBTM(num_topics=2, V=vocab)
for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
    biterms_chunk = biterms[i:i + 100]
    btm.fit(biterms_chunk, iterations=50)
    topics = btm.transform(biterms)
    
print("\n\n Topic coherence ..")
topic_summuary(btm.phi_wz.T, apple_vectorized, vocab, 15)

100%|██████████| 50/50 [00:00<00:00, 706.16it/s]




 Topic coherence ..
Topic 0 | Coherence=-330.54 | Top words= music microsoft help app download store iamtbotouch give iphone tim google podcast cook pro itunes
Topic 1 | Coherence=-340.13 | Top words= music spotify song app year phone back playlist tidal new video first store last lot
Average topic coherence for the top words is -335.3335452035808


{'coherence': [-330.53707906334904, -340.1300113438126],
 'top_words': [array(['music', 'microsoft', 'help', 'app', 'download', 'store',
         'iamtbotouch', 'give', 'iphone', 'tim', 'google', 'podcast',
         'cook', 'pro', 'itunes'], dtype='<U15'),
  array(['music', 'spotify', 'song', 'app', 'year', 'phone', 'back',
         'playlist', 'tidal', 'new', 'video', 'first', 'store', 'last',
         'lot'], dtype='<U15')]}

In [37]:
count_0=0
for i in range(len(tweet_apple_words)):
    if topics[i].argmax()==0:
        count_0+=1
print('Number of tweets under Topic 0: ', count_0)
count_1=0
for i in range(len(tweet_apple_words)):
    if topics[i].argmax()==1:
        count_1+=1
print('Number of tweets under Topic 1: ', count_1)

Number of tweets under Topic 0:  1726
Number of tweets under Topic 1:  485


## Huawei

In [23]:
tweet_huawei_clean=tweet_combined_clean[tweet_combined_clean['brand']==2]
tweet_huawei_words=tweet_huawei_clean['text'].apply(text_processer)
huawei_vectorized=cv.fit_transform(tweet_huawei_words).toarray()

In [28]:
vocab = np.array(cv.get_feature_names())
biterms = vec_to_biterms(huawei_vectorized)

btm = oBTM(num_topics=2, V=vocab)
for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
    biterms_chunk = biterms[i:i + 100]
    btm.fit(biterms_chunk, iterations=50)
    topics = btm.transform(biterms)
    
print("\n\n Topic coherence ..")
topic_summuary(btm.phi_wz.T, huawei_vectorized, vocab, 15)

100%|██████████| 50/50 [00:00<00:00, 4558.43it/s]




 Topic coherence ..
Topic 0 | Coherence=-262.76 | Top words= china ccp chinese red claw connection dragon expose focus globalnews dmhdpxfuci ttsampaio mask communist chinadaily
Topic 1 | Coherence=-374.29 | Top words= china company chinese canada europe people mask leeszla world pro globalnews phone ccp mike psyberchic
Average topic coherence for the top words is -318.529739420865


{'coherence': [-262.76479176544495, -374.2946870762851],
 'top_words': [array(['china', 'ccp', 'chinese', 'red', 'claw', 'connection', 'dragon',
         'expose', 'focus', 'globalnews', 'dmhdpxfuci', 'ttsampaio', 'mask',
         'communist', 'chinadaily'], dtype='<U15'),
  array(['china', 'company', 'chinese', 'canada', 'europe', 'people',
         'mask', 'leeszla', 'world', 'pro', 'globalnews', 'phone', 'ccp',
         'mike', 'psyberchic'], dtype='<U15')]}

In [29]:
count_0=0
for i in range(len(tweet_huawei_words)):
    if topics[i].argmax()==0:
        count_0+=1
print('Number of tweets under Topic 0: ', count_0)
count_1=0
for i in range(len(tweet_huawei_words)):
    if topics[i].argmax()==1:
        count_1+=1
print('Number of tweets under Topic 1: ', count_1)

Number of tweets under Topic 0:  1198
Number of tweets under Topic 1:  1206
